# Lao Part-of-speech

I use Lao corpus from https://github.com/FoVNull/SeqLabeling.

In [1]:
!wget https://github.com/FoVNull/SeqLabeling/raw/main/corpus/Lao/Lao_train.txt
!wget https://github.com/FoVNull/SeqLabeling/raw/main/corpus/Lao_test.pkl
!wget https://github.com/FoVNull/SeqLabeling/raw/main/corpus/Lao/Lao_valid.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [22]:
import pickle
from pythainlp.tag import PerceptronTagger
from sklearn import metrics
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

In [3]:
with open("Lao_test.pkl","rb") as f:
    test_data = pickle.load(f)

In [4]:
alltag = []

In [5]:
def readfile(name):
    with open(name ,"r",encoding="utf-8-sig") as f:
        _data = [i.strip() for i in f.readlines()]
    data = []
    for i in _data:
        _t = []
        _j = i.split("\t")
        for j in _j:
            _w, _tag = j.split("\u200b")
            if _w=="":
                _w = "<space>"
            if _tag != '':
                _t.append((_w,_tag))
                alltag.append(_tag)
        data.append(_t)
    return data

In [6]:
train_dataset = readfile("Lao_train.txt")

In [7]:
valid_dataset = readfile("Lao_valid.txt")

In [8]:
alltag = list(set(alltag))

In [9]:
valid_x = [[i[0] for i in j] for j in valid_dataset]
valid_y = [[i[1] for i in j] for j in valid_dataset]

In [10]:
filename = "ptagger_SeqLabeling_corpus.json"

In [11]:
tagger = PerceptronTagger()

In [12]:
tagger.train(train_dataset, save_loc=filename)

In [13]:
tagger.tag(valid_x[0])

[('ບ່ອນໃດ', 'ADV'),
 ('ສາວ', 'N'),
 ('ງາມ', 'N'),
 ('ຫລາຍ', 'N'),
 ('ລະ', 'ADV'),
 ('ຮຽນ', 'V'),
 ('ໂລດ', 'N'),
 ('ບໍ່', 'NEG'),
 ('ສົນ', 'V'),
 ('ກັບ', 'PRE'),
 ('ຫຍັງ', 'DBQ'),
 ('ເລີຍ', 'ADV'),
 ('ອິໆໆໆ', 'IAC')]

In [14]:
valid_y[0]

['CLF',
 'N',
 'V',
 'N',
 'ADV',
 'V',
 'N',
 'NEG',
 'ADJ',
 'PRE',
 'NTR',
 'ADV',
 'IAC']

In [15]:
pred = [[t for _,t in tagger.tag(sent)] for sent in valid_x]
gold = valid_y

In [16]:
for i in range(0,len(pred)):
    if len(pred[i]) != len(gold[i]):
        print(i)
        print(pred[i],gold[i])

In [17]:
len(pred)

1154

In [18]:
len(gold)

1154

In [19]:
alltag

['PUNCT',
 'PRA',
 'N',
 'CNM',
 'FIX',
 'TTL',
 'NTR',
 'PRN',
 'DMN',
 'CLF',
 'ONM',
 'DBQ',
 'COJ',
 'IAC',
 'ADJ',
 'IBQ',
 'DAN',
 'PRE',
 'NEG',
 'PRS',
 'ADV',
 'DAQ',
 'IAQ',
 'PVA',
 'V',
 'REL']

In [23]:
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    
    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [26]:
print(bio_classification_report(gold, pred))

              precision    recall  f1-score   support

         ADJ       0.81      0.75      0.78       879
         ADV       0.92      0.84      0.87       584
         CLF       0.92      0.82      0.87       345
         CNM       0.92      0.70      0.79        93
         COJ       0.93      0.87      0.90       861
         DAN       0.00      0.00      0.00         3
         DAQ       1.00      1.00      1.00         3
         DBQ       0.79      0.78      0.78        49
         DMN       1.00      1.00      1.00       164
         FIX       0.95      0.90      0.92        97
         IAC       0.98      0.98      0.98       109
         IAQ       1.00      1.00      1.00        10
         IBQ       1.00      1.00      1.00         1
           N       0.78      0.90      0.83      5108
         NEG       0.98      0.96      0.97       207
         NTR       0.95      0.94      0.95       149
         ONM       0.00      0.00      0.00         6
         PRA       0.90    

c:\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
